# Запросы к Elasticsearch из Javascript
Этот пример показывает как делать запросы к Elasticsearch из Javascript.

In [7]:
%%html
<div id="chart-response" style="height:400px; border: 1px dashed silver;" ></div>

In [5]:
%%javascript

require(['https://cdn.plot.ly/plotly-latest.min.js'],(Plotly)=>{
    
    // Конечная точка Elasticsearch 
    const url = "https://log-monitor.rg.ru/elasticsearch/_sql?format=json"
    
    // Запрос который будем выполнять
    const query = { query: `
    SELECT HISTOGRAM("@timestamp", INTERVAL 30 SECOND) as interval, 
    count(*) as count
    FROM "log-generator-logrus*"
    GROUP by interval
    ORDER BY interval DESC
    LIMIT 30        
    ` }
 
    
    // выполняет запрос к эластик
    async function doQuery(url, query) {
        var res = await fetch(url, { 
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify(query),
        })
        var data = await res.json()
        showPlot(document.getElementById('chart-response'), data)
    }

    // строит график
    function showPlot(containerElement, data) {
        var rows = data['rows']
        var xx = rows.map(r => r[0])
        var yy = rows.map(r => r[1]) 
        var d = [{ x: xx, y: yy, text: yy, textposition: 'auto', type: 'bar' }]
        var layout = {
            title: 'Количество записей лога в 30-ти сек интервалах ',
        }
        Plotly.newPlot( containerElement, d , layout )
    }
   
    // делаем запросы каждые 3 сек
    setInterval(()=> doQuery(url,query), 3000)

})

<IPython.core.display.Javascript object>